In [1]:
# %tensorflow_version 2.x
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from tensorflow.python.framework.ops import disable_eager_execution
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.regularizers import l2
import random
disable_eager_execution()


In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [4]:
base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')

# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

In [20]:
path_train = train_dir
path_test = validation_dir
Batch_size = 64
trdata = ImageDataGenerator(preprocessing_function=preprocess_input)
traindata = trdata.flow_from_directory(directory=path_train,target_size=(224,224), batch_size=Batch_size)

val_data = ImageDataGenerator(preprocessing_function=preprocess_input)
validation_data = val_data.flow_from_directory(directory=path_test, target_size=(224,224), batch_size=Batch_size)


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [9]:
base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

Instructions for updating:
Colocations handled automatically by placer.
17227776/17225924 [==============================] - 0s 0us/step


In [21]:
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
preds=Dense(2,activation='softmax')(x) #final layer with softmax activation

In [22]:
model=Model(inputs=base_model.input,outputs=preds)
#specify the inputs
#specify the outputs
#now a model has been created based on our architecture

In [23]:
opt = SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer=opt, loss=tf.keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [24]:
## calbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, verbose=1)
model_checkpoint_callback1 = ModelCheckpoint(
    filepath='./model/best_checkpoint',
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)


In [25]:
! mkdir model

mkdir: cannot create directory ‘model’: File exists


In [27]:
hist = model.fit(traindata, validation_data=validation_data, epochs=10, steps_per_epoch=int(traindata.samples/Batch_size), callbacks=[reduce_lr, model_checkpoint_callback1])

Epoch 1/10
31/31 [==============================] - 15s 493ms/step - batch: 15.0000 - size: 62.4516 - loss: 0.0089 - accuracy: 0.9969 - val_loss: 0.0658 - val_accuracy: 0.9780 - lr: 1.0000e-07
Epoch 2/10
31/31 [==============================] - 15s 497ms/step - batch: 15.0000 - size: 62.4516 - loss: 0.0084 - accuracy: 0.9979 - val_loss: 0.0646 - val_accuracy: 0.9780 - lr: 1.0000e-07
Epoch 3/10
31/31 [==============================] - 15s 498ms/step - batch: 15.0000 - size: 62.4516 - loss: 0.0091 - accuracy: 0.9969 - val_loss: 0.0639 - val_accuracy: 0.9800 - lr: 1.0000e-07
Epoch 4/10
31/31 [==============================] - ETA: 0s - batch: 15.0000 - size: 62.4516 - loss: 0.0119 - accuracy: 0.9928
Epoch 00004: ReduceLROnPlateau reducing learning rate to 9.999998695775504e-09.
31/31 [==============================] - 16s 509ms/step - batch: 15.0000 - size: 62.4516 - loss: 0.0119 - accuracy: 0.9928 - val_loss: 0.0638 - val_accuracy: 0.9800 - lr: 1.0000e-07
Epoch 5/10
31/31 [==============

In [28]:
mobilenet_topless = MobileNet(weights='imagenet', include_top=False, pooling='avg')
mobilenet_topless.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, None, None, 

In [29]:
img_path = 'elephant.JPEG'
img = image.load_img(img_path, target_size=(224, 224))
img_data = image.img_to_array(img)
img_data = np.expand_dims(img_data, axis=0)
img_data = preprocess_input(img_data)

mobilenet_feature = mobilenet_topless.predict(img_data)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


In [31]:
print(mobilenet_feature, mobilenet_feature.shape)

[[1.4634607e+00 0.0000000e+00 3.4115078e+00 ... 2.1817931e-03
  3.9770585e-01 6.5536790e-02]] (1, 1024)


In [33]:
layer_name = 'conv_pw_2'
intermediate_layer_model = Model(inputs=mobilenet_topless.input,
                                 outputs=mobilenet_topless.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict(img_data)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


In [35]:
print(intermediate_output, intermediate_output.shape)

[[[[-6.898582    0.16693354  0.21672018 ...  0.39833194 -0.3577439
    -1.1823068 ]
   [-7.176694   -0.08222113  0.3484465  ...  0.31386063 -0.18230431
    -0.9712083 ]
   [-7.1079893  -0.09013498  0.05796991 ...  0.51744777 -0.22997679
    -1.0046074 ]
   ...
   [-7.1391664  -0.16828789  0.4661972  ...  0.33083448 -0.42511734
     0.3218016 ]
   [-7.281604   -0.00931009  0.10628615 ...  0.34451562 -0.6007021
     0.47500598]
   [-5.577486    1.6338406   0.39726943 ...  0.45652598 -0.20085168
     0.6486339 ]]

  [[-7.026533    0.12801538  0.3230431  ...  0.1392549  -0.64967984
    -1.0446903 ]
   [-6.416327   -0.323837   -0.02983422 ...  0.32237428 -0.6207523
    -0.93541694]
   [-6.5229225  -0.2314368  -0.04065886 ...  0.3614157  -0.5672194
    -0.8640842 ]
   ...
   [-6.640675   -0.37015325  0.37367377 ...  0.52493906 -0.7358516
     0.32671538]
   [-6.634864   -0.21942906  0.07716984 ...  0.35733894 -0.9423119
     0.43597075]
   [-5.518675    1.5136993   0.19621639 ...  0.4167543 